In [10]:

from TraceLib.my_trace import Trace
from Proto.model_pb2 import Trace
from Proto.query_pb2_grpc import QueryService

a = Trace()
b = QueryService()
b.FindTraces()

ModuleNotFoundError: No module named 'TraceLib'

In [1]:
from typing import List, Dict
import json
import requests
import time

def build_jaeger_trace_endpoint(params : List[str]) -> str:
    return f"http://localhost:6686/api/traces?{"&".join(params)}"

def build_jaeger_spans_endpoint_url(limit: int, service : str, namespace : str, operation : str, lookback_minutes : int, tags : Dict[str,str]) -> str:
    lookback_seconds = 60*lookback_minutes
    start_ms = int((time.time() - lookback_seconds)*1_000_000)
    end_ms = int(time.time()*1_000_000)
    
    tags["service.namespace"] = namespace
    full_tags = "%2C".join([f'"{key}"%3A"{value}"' for (key,value) in tags.items()])
    limit_str = f"limit={limit}"
    lookback_str = f"lookback={lookback_seconds}s"
    service_str = f"service={service}"
    operation_str = f"operation={operation}"
    start_str = f"start={start_ms}"
    end_str = f"end={end_ms}"
    tags_str = f"tags=%7B{full_tags}%7D"
    return build_jaeger_trace_endpoint([limit_str,lookback_str,service_str,operation_str,start_str,end_str,tags_str])

def scrape_jaeger_endpoint(url : str):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        raise err

    print(f"   Response received. Processing json")
    return json.loads(response.text)


In [2]:
url = build_jaeger_spans_endpoint_url(10,"digitiser-aggregator", "pipeline_1", "Frame", 60,dict());
print(url)
value = scrape_jaeger_endpoint(url)
print(value)

http://localhost:6686/api/traces?limit=10&lookback=3600s&service=digitiser-aggregator&operation=Frame&start=1733073324024038&end=1733076924024039&tags=%7B"service.namespace"%3A"pipeline_1"%7D
   Response received. Processing json
{'data': [{'traceID': '82ded03a74ec5ea3483b5280910bc8d4', 'spans': [{'traceID': '82ded03a74ec5ea3483b5280910bc8d4', 'spanID': 'b18bff7e03b69db2', 'operationName': 'Frame', 'references': [], 'startTime': 1733076921666518, 'duration': 2007967, 'tags': [{'key': 'otel.library.name', 'type': 'string', 'value': 'opentelemetry-otlp'}, {'key': 'otel.library.version', 'type': 'string', 'value': '0.15.0'}, {'key': 'code.filepath', 'type': 'string', 'value': 'digitiser-aggregator/src/frame/partial.rs'}, {'key': 'code.namespace', 'type': 'string', 'value': 'digitiser_aggregator::frame::partial'}, {'key': 'code.lineno', 'type': 'int64', 'value': 79}, {'key': 'thread.id', 'type': 'int64', 'value': 1}, {'key': 'thread.name', 'type': 'string', 'value': 'main'}, {'key': 'metad

In [22]:
for v in value['data']:
    for k in v['spans']:
            print([a['value'] for a in k['tags'] if 'metadata_timestamp' in a['key']])

['2024-12-01T18:15:21.616897180+00:00']
['2024-12-01T18:15:21.616897180+00:00', '2024-12-01T18:15:21.616897180+00:00']
['2024-12-01T18:15:21.616897180+00:00', '2024-12-01T18:15:21.616897180+00:00']
['2024-12-01T18:15:21.616897180+00:00', '2024-12-01T18:15:21.616897180+00:00']
['2024-12-01T18:15:21.616897180+00:00', '2024-12-01T18:15:21.616897180+00:00']
['2024-12-01T18:15:21.616897180+00:00', '2024-12-01T18:15:21.616897180+00:00']
['2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01T18:15:21.596897100+00:00']
['2024-12-01T18:15:21.596897100+00:00', '2024-12-01

In [ ]:

url = build_jaeger_spans_endpoint_url(10,"digitiser-aggregator", "pipeline_1", "Frame", 60, {"frame_is_expired": "true"});
spans = scrape_jaeger_endpoint(url)

for span in spans:
    num_digitisers = len(span["children"])
    timestamp = spans["tags"]["metadata_timestamp"]

    url = build_jaeger_spans_endpoint_url(10,"time-to-events", "pipeline_1", "process_digitiser_message", 60, {"metadata_timestamp": timestamp});
    spans = scrape_jaeger_endpoint(url)
    print(f"num digitisers in aggregator frame: {num_digitisers}, num digitisers in event formation: {len(spans)}")